In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import time 

import torch
import mrmr
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [2]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'smartphone+based+recognition+of+human+activities+and+postural+transitions/'
MODELS_PATH = DATA_PATH + 'models/raw-models/'

In [3]:
# Load the data
train_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data/train-data.txt", sep='\s+')
test_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data/test-data.txt", sep='\s+')

train_set.columns=['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]
test_set.columns=['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]

# Divide dataset to train and test set
print(train_set.shape)
print(test_set.shape)

(570908, 8)
(244650, 8)
